In [1]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
import re
from time import sleep as time_sleep, time
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED
from selenium import webdriver
import requests
from webdriver_manager.chrome import ChromeDriverManager
import lxml
from itertools import chain
from typing import NamedTuple
import pickle

In [2]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    href: str

In [3]:
driver = webdriver.Chrome()

# Получение адресов городов

# Получение ссылок на Магазины

## Получение ссылки на старицы каталогов с магазинами

### Функции

In [7]:
def _get_page_count(drive: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=20) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@class='i-block-helper uid-e710417b6f b-dsk-paginator p-dsk-srch-retailers__paginator uid-6d736fe587 b-dsk-paginator_disabled_false b-dsk-paginator_progress_false']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', class_='i-block-helper uid-e710417b6f b-dsk-paginator p-dsk-srch-retailers__paginator uid-6d736fe587 b-dsk-paginator_disabled_false b-dsk-paginator_progress_false')
        except Exception as ex:
            print(ex)
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            print(ex)
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise Exception("timeout")
    
    return out_list

### Выполнение

In [5]:
# driver.get('https://edadeal.ru/lipeck/retailers')
url = 'https://edadeal.ru/lipeck/retailers'

In [8]:
temp_url_list = _get_page_count(driver, url, '?page=')
temp_url_list

['https://edadeal.ru/lipeck/retailers?page=1',
 'https://edadeal.ru/lipeck/retailers?page=2',
 'https://edadeal.ru/lipeck/retailers?page=3',
 'https://edadeal.ru/lipeck/retailers?page=4',
 'https://edadeal.ru/lipeck/retailers?page=5']

## Получение непосредственно все ссылки на Магазины

### Функции

In [9]:
def _get_retailers_links(drive: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )

    
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
        except Exception as ex:
            print(ex)
            continue
        
        try:    
            for item in f1:
                href = host + item.find('a').get('href')
                title = item.find('a').find('h4').get('title')
                discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
                out_list.append(market_link(sity, title, discounts, href))        
                
        except Exception as ex:
            print(ex)
            continue
        
        break
    
    else:
        raise Exception('timeout')
    
    return out_list

### Выполнение

In [10]:
temp_retailer_links = list()
for link in temp_url_list:
    temp_retailer_links.append(_get_retailers_links(driver, link, 'lipetsk'))

retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))
print(retailer_links)

55
[market_link(sity='lipetsk', title='Магнит', discounts=10400, href='https://edadeal.ru/lipeck/retailers/magnit-univer'), market_link(sity='lipetsk', title='Пятёрочка', discounts=30, href='https://edadeal.ru/lipeck/retailers/5ka'), market_link(sity='lipetsk', title='Магнит Косметик', discounts=10303, href='https://edadeal.ru/lipeck/retailers/magnit-cosmo'), market_link(sity='lipetsk', title='Перекрёсток', discounts=1877, href='https://edadeal.ru/lipeck/retailers/perekrestok'), market_link(sity='lipetsk', title='Чижик', discounts=15, href='https://edadeal.ru/lipeck/retailers/chizhik'), market_link(sity='lipetsk', title='Европа', discounts=737, href='https://edadeal.ru/lipeck/retailers/europa-ts'), market_link(sity='lipetsk', title="О'кей гипермаркет", discounts=851, href='https://edadeal.ru/lipeck/retailers/okmarket-giper'), market_link(sity='lipetsk', title='Сытый Бацька', discounts=12, href='https://edadeal.ru/lipeck/retailers/sytyi_batka'), market_link(sity='lipetsk', title='Липка'

#### Save/Load

In [192]:
with open('data/retailer_links.pickle', 'wb') as file:
    pickle.dump(retailer_links, file)


In [11]:
with open('data/retailer_links.pickle', 'rb') as file:
    retailer_links = pickle.load(file)

# Получение адресов Товаров

In [12]:
retailer_links

[market_link(sity='lipetsk', title='Магнит', discounts=10427, href='https://edadeal.ru/lipeck/retailers/magnit-univer'),
 market_link(sity='lipetsk', title='Пятёрочка', discounts=30, href='https://edadeal.ru/lipeck/retailers/5ka'),
 market_link(sity='lipetsk', title='Перекрёсток', discounts=1877, href='https://edadeal.ru/lipeck/retailers/perekrestok'),
 market_link(sity='lipetsk', title='Магнит Косметик', discounts=1037, href='https://edadeal.ru/lipeck/retailers/magnit-cosmo'),
 market_link(sity='lipetsk', title="О'кей гипермаркет", discounts=853, href='https://edadeal.ru/lipeck/retailers/okmarket-giper'),
 market_link(sity='lipetsk', title='Лента Гипермаркет', discounts=5799, href='https://edadeal.ru/lipeck/retailers/lenta-giper'),
 market_link(sity='lipetsk', title='Победа', discounts=82, href='https://edadeal.ru/lipeck/retailers/pobeda'),
 market_link(sity='lipetsk', title='Чижик', discounts=15, href='https://edadeal.ru/lipeck/retailers/chizhik'),
 market_link(sity='lipetsk', title=